# Model Experimentation
This notebook is designed to streamline experimentation with different models.

1. Selecting a model, dataset, tracker and hyperparameters
2. Training the model and evaluating both object detection and tracking performance
3. Saving the results in wandb
4. Storing the model and evaluation annotations in a folder
5. **Investigation** of the results, detection of failure cases and model improvement
6. **Iterating** over steps 1-5



In [18]:
from architectures import yolov8
# import wandb

> run `wandb login` in terminal before running this notebook

### Select a model, dataset, tracker and hyperparameters

In [ ]:
architectures = {
  "YOLOv8": YOLOv8
  }   # map model name to model class
dataset_mapping = {} # map dataset name to dataset class

In [ ]:
def model_experimentation(hyperparameters):
  run = wandb.init(project="SharkTrack", config=hyperparameters)

  model_name = hyperparameters['model_name']
  dataset_name = hyperparameters['dataset_name']

  model = model_mapping[model_name]
  dataset = dataset_mapping[dataset_name]

  model.fit(dataset)
  mAP, f1, mota, motp, idf1, graph, annotations =  model.evaluate(dataset)

  wandb.log({"mAP": mAP, "F1": f1, "MOTA": mota, "MOTP": motp, "IDF1": idf1})

  # also save path where store model and annotations

  # Save model and annotations in hyperparameters[save_path]
